In [2]:
import time
import os
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from siriuspy.search import IDSearch
from siriuspy.devices import APU, SOFB, HLFOFB, IDFF, Tune, DVFImgProc

import epics
from mathphys.functions import save, load

timeout = 3

# Functions definition

In [2]:
def initialize_apu(beamline):
    # Search ID
    devnameapu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    apu = APU(devname=devnameapu)

    # Disable beamline control
    apu.cmd_beamline_ctrl_disable()
    print('beamline control: ', apu.is_beamline_ctrl_enabled)


    # Set gap speed
    apu.set_phase_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(apu.gap_speed))

    # Set gap to parked condition
    apu.set_phase(apu.kparameter_parked)
    time.sleep(0.5)
    print('phase: {:.3f} mm'.format(apu.phase))

    return apu

def move_apu_phase(apu:APU, phase, timeout, verbose=False):
    apu.set_phase(phase)
    time.sleep(0.5)
    print('Phase-RB {:.3f} mm'.format(apu.phase)) if verbose else 0
    if apu.cmd_move_kparameter_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while apu.is_moving:
            time.sleep(0.1)
            print('Current phase {:.3f} mm.'.format(apu.phase_mon), end='\r') if verbose else 0
        print('Phase {:.3f} mm reached.'.format(apu.phase)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def get_CAX_data(tune, imgproc, nr_acq=10, acq_sleep=0.5):
        """Get data.

        Returns:
            data: Numpy 1d array
        """
        sigmax = np.zeros(nr_acq)
        sigmay = np.zeros(nr_acq)
        sigma1 = np.zeros(nr_acq)
        sigma2 = np.zeros(nr_acq)
        angle = np.zeros(nr_acq)
        tunex = np.zeros(nr_acq)
        tuney = np.zeros(nr_acq)
        images = np.zeros((nr_acq, imgproc.image_sizey, imgproc.image_sizex))
        for i in range(nr_acq):
            sigmax[i] = imgproc.roix_fit_sigma
            sigmay[i] = imgproc.roiy_fit_sigma
            sigma1[i] = imgproc.fit_sigma1
            sigma2[i] = imgproc.fit_sigma2
            angle[i] = imgproc.fit_angle
            tunex[i] = tune.tunex
            tuney[i] = tune.tuney
            images[i] = imgproc.image
            time.sleep(acq_sleep)

        img_params = np.zeros((7, nr_acq))
        img_params[0, :] = sigmax
        img_params[1, :] = sigmay
        img_params[2, :] = sigma1
        img_params[3, :] = sigma2
        img_params[4, :] = angle
        img_params[5, :] = tunex
        img_params[6, :] = tuney
        return img_params, images


## Search devnames

In [4]:
devnameapu = IDSearch.conv_beamline_2_idname(beamline='SAPUCAIA')

## Initialize devices

In [ ]:
apu = initialize_apu(beamline='SAPUCAIA')
apu.set_phase_speed(0.5)
time.sleep(0.5)
print(apu.phase_speed)

In [ ]:
tune = Tune(Tune.DEVICES.SI)
imgproc = DVFImgProc(DVFImgProc.DEVICES.CAX_DVF2)

imgproc.wait_for_connection(timeout=timeout)
print('IMG Proc connected: ', imgproc.connected)

tune.wait_for_connection(timeout=timeout)
print('Tune connected: ', tune.connected)


In [ ]:
sofb = SOFB(SOFB.DEVICES.SI)
fofb = HLFOFB(HLFOFB.DEVICES.SI)

### Make sure SOFB and FOFB loops are opened

In [ ]:
sofb.cmd_turn_off_autocorr()
sofb.cmd_change_opmode_to_sloworb(timeout=timeout)
fofb.cmd_turn_off_loop_state()

# Meas orbit distortions

In [ ]:
phases = np.arange(11, 0, -1)
print(phases)

In [ ]:
data = dict()
data['phase'] = list()
data['timestamps'] = list()
data['orbx'] = list()
data['orby'] = list()
data['phase_speed'] = list()
data['sigmax'] = list()
data['sigmay'] = list()
data['sigma1'] = list()
data['sigma2'] = list()
data['angle'] = list()
data['tunex'] = list()
data['tuney'] = list()
data['CAX_images'] = list()
t0 = time.time()
for i, phase in enumerate(phases):

    # Move gap
    sucess = move_apu_phase(apu, phase=phase, timeout=timeout, verbose=True)
    
    # Verify if undulator moved
    if not sucess:
        break

    # Wait orbit distortion to become stationary
    time.sleep(3)
    
    # Record orbit and other info
    data['phase'].append(phase)
    data['timestamps'].append(time.time())
    data['orbx'].append(sofb.orbx)
    data['orby'].append(sofb.orby)
    data['phase_speed'].append(apu.phase_speed)
    data['beamline'] = IDSearch.conv_idname_2_beamline(apu.devname)

    # Record CAX img params and tune
    caxdata, images = get_CAX_data(tune, imgproc, nr_acq=10, acq_sleep=0.5)
    data['sigmax'].append(caxdata[0, :])
    data['sigmay'].append(caxdata[1, :])
    data['sigma1'].append(caxdata[2, :])
    data['sigma2'].append(caxdata[3, :])
    data['angle'].append(caxdata[4, :])
    data['tunex'].append(caxdata[5, :])
    data['tuney'].append(caxdata[6, :])
    data['CAX_images'].append(images)

print(f'Done! ETA: {time.time()-t0:.3f}s')
save(data, 'orbit_distortions_CAX_IVU_{}_iter0.pickle'.format(IDSearch.conv_idname_2_beamline(ivu.devname)))